# Synchronous Multiple View

This notebook is an implementation, debugging and analysis of a Multiple View Synchronous Motion Capture System. 

The goal is to implement and analyse, how all of the parameters and models used will affect the overall triangulation. For this:
1. Instanciate a set of Vision Sensors in both *CoppeliaSim*;
2. Build Fundamental Matrices for each pair of instanciated cameras;
3. Choose a pair for triangulation;
4. In simulation, get images for each PTS and triangulate the points in each one;
5. Check Real Time Factor, Triangulation Error and Jitter. 

---

In [14]:
# Importing modules...
import numpy as np
import cv2
import time

import sys
sys.path.append('../..') # Go back to base directory

from modules.vision.camera import Camera
from modules.vision.epipolar_geometry import build_essential_matrix, build_fundamental_matrix, order_centroids
from modules.vision.blob_detection import detect_blobs

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

# Init client 
client = RemoteAPIClient()    # Client object 
sim = client.getObject('sim') # Simulation object

# Instanciating Camera Objects

To generate the data regarding the mathematical model of the vision sensor in *CoppeliaSim*, it will be instanciated a Camera Obejcts that matches the Vision Sensors' intrinsic and extrinsic parameters. 

---

In [15]:
n_cameras = 8
camera = [] # Camera objects list

for ID in range(n_cameras):
    # Get the vision sensor handle
    vision_sensor_handle = sim.getObject(f'/VisionSensor[{ID}]')

    # Get Coppelia's 3x4 object matrix
    object_matrix = np.array(sim.getObjectMatrix(vision_sensor_handle)).reshape((3,4))

    # X and Y axis of Coppelia's Vision Sensor are inverted
    object_matrix[:,0] *= -1 # Invert x vector column
    object_matrix[:,1] *= -1 # Invert y vector column

    camera.append(Camera(# Simulation handling
                          vision_sensor_handle=vision_sensor_handle,
   
                          # Intrinsic Parameters
                          resolution=(720,720), 
                          fov_degrees=60.0,     
          
                          # Extrinsic Parameters
                          object_matrix=object_matrix,
          
                          # Rational Lens Distortion Model
                          # distortion_model='rational',
                          # distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]),

                          # Fisheye Lens Distortion Model
                          distortion_model='fisheye',
                          distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
          
                          # Image Noise Model
                          snr_dB=13
                          ))

# Computing Fundamental Matrices

For each pair $i, j$, a fundamental matrix will be calculated between camera's $i$ and $j$. In this case, this information will be storted in a matrix $F$ such that $F_{ij}$ will be the fundamental matrix between camera's $i$ and $j$.

---

In [16]:
fundamental_matrix = np.array(np.zeros((n_cameras, n_cameras, 3, 3)))

for reference in range(n_cameras):
    for auxiliary in range(n_cameras):
        if reference == auxiliary:
            continue

        E = build_essential_matrix(camera[reference].extrinsic_matrix, camera[auxiliary].extrinsic_matrix)

        F = build_fundamental_matrix(camera[reference].intrinsic_matrix, camera[auxiliary].intrinsic_matrix, E)

        fundamental_matrix[reference][auxiliary] = F

# Running Simulation

For a running simulation, in each Presentation Timestamp (PTS), the following process will happen:

1. Get images of all cameras;
2. Detect blobs;
3. Undistort detected blob coordinates;
4. Order blobs based on epipolar lines;
5. Triangulate markers.

For each PTS the data will be saved for later analysis.

To evaluate how well the system is able to process at run-time, the Real Time Factor of the system will be calculated:

$${RTF} = \frac{t_S}{t_R}$$

Where $t_S$ is the total simulation time, and $t_R$ is the total ammount of real time that the code took to ran.

---

In [17]:
# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default.
# Setting the idle loop to run at full speed for this program
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)   
sim.setInt32Param(sim.intparam_idle_fps, 0)

pair = (0, 1)
reference, auxiliary = pair


simulation_time = 5 # In seconds
images = {} # Images of all cameras in each presentation timestamps
marker_positions = {} # Triangulated marker positions in each presentation timestamps

# Simulation begins here
sim.startSimulation()

timer_start = time.time()

while (t := sim.getSimulationTime()) < simulation_time:
    # Get and save images
    sync_images = [camera[ID].get_coppelia_image(sim.getVisionSensorCharImage) for ID in range(n_cameras)]
    images[t] = sync_images    

    # Detect blobs on the distorted image
    distorted_blobs = [detect_blobs(sync_images[ID]) for ID in pair]

    # If no blobs were detected then images are invalid
    if any(blob is None for blob in distorted_blobs):
        continue # Jump to next iteration
        
    # Undistort blobs
    undistorted_blobs = [camera[ID].undistort_points(distorted_blobs[ID].T.reshape(1, -1, 2).astype(np.float32), 
                                                     camera[ID].intrinsic_matrix, 
                                                     camera[ID].distortion_coefficients,
                                                     np.array([]),
                                                     camera[ID].intrinsic_matrix).reshape(-1,2).T for ID in pair]

    # Order blobs
    epilines_auxiliary = cv2.computeCorrespondEpilines(points=undistorted_blobs[reference].T, 
                                                       whichImage=1, 
                                                       F=fundamental_matrix[reference][auxiliary]).reshape(-1,3)
    
    undistorted_blobs[auxiliary] = order_centroids(undistorted_blobs[auxiliary], epilines_auxiliary)

    # Triangulate markers
    triangulated_positions = cv2.triangulatePoints(camera[reference].projection_matrix.astype(float), 
                                                   camera[auxiliary].projection_matrix.astype(float), 
                                                   undistorted_blobs[reference].astype(float), 
                                                   undistorted_blobs[auxiliary].astype(float))
    
    # Normalize homogeneous coordinates and save
    marker_positions[t] = (triangulated_positions / triangulated_positions[-1])[:-1, :]

timer_end = time.time()

# Closing all open windows 
cv2.destroyAllWindows()

# Restore the original idle loop frequency:
sim.setInt32Param(sim.intparam_idle_fps, defaultIdleFps)

# Simulation ends here
sim.stopSimulation()

print(f'Real Time Factor: {simulation_time / (timer_end - timer_start)}')

Real Time Factor: 0.4302817176659904


# Analysing Precision and Accuracy 

Since the ground truth data of the marker's position is available, it's possible to evaluate the Precision and Accuracy of the system for the capture of a single stationary marker on the scene.

Calculating the average position of the marker and evaluating the difference between it's ground truth will determine the **Precision**. Calculating the standard deviation of the marker's position will determine how accurate the system is by it's **Jitter**.

---

In [18]:
# Calculate Accuracy and Precision 
all_marker_positions = None
for frame, pts in enumerate(marker_positions.keys()):
    if all_marker_positions is None:
        all_marker_positions = marker_positions[pts]
    else:
        all_marker_positions = np.hstack((all_marker_positions, marker_positions[pts]))

true_position = np.array(sim.getObjectPosition(sim.getObject('/Marker')))
average_position = np.mean(all_marker_positions, axis=1)
std_dev_position = np.std(all_marker_positions, axis=1)

print('Ground Truth Position:', true_position)
print('Average Position:', average_position)
print(f'Triangulation Error: {np.linalg.norm(average_position - true_position) * 1e3 :.2f} mm')
print(f'Standard Deviation: {np.linalg.norm(std_dev_position) * 1e3 :.2f} mm')

Ground Truth Position: [0.  0.  0.1]
Average Position: [ 0.00360371 -0.00148805  0.09964503]
Triangulation Error: 3.91 mm
Standard Deviation: 1.76 mm


# Playing Back the Camera's Feed

The following cell will replicate a real time camera feed of the simulation. Change the `ID` parameter to switch between camera views.

---

In [19]:
# Playback a camera image feed in real time
ID = 0 # Camera ID to be watched
timestep = int(1e3 * simulation_time/len(images.keys()))
for frame, pts in enumerate(images.keys()):
    cv2.imshow(f'Camera {ID}', images[pts][ID])
    cv2.waitKey(timestep)

# Closing all open windows 
cv2.destroyAllWindows()